In [ ]:
import numpy as np
import pandas as pd
import time
import math
import matplotlib.pyplot as plt
import seaborn as sns
from decimal import Decimal

In [1]:
def sim_plasticity_evo(pop_size_init = 500, num_generations = 400, r0 = 2.15, b_init = 0.25, x_init = 9.0, beta = 1.0, K = 1000, v = None, v_array = None, omegaB = 9.0, omegaZ = 25.0, sigmaB = 1.0, sigmaX = 1.0, sigmaE = 1.0):
    # Reaction Norm Arrays
    x_trait = np.zeros(num_generations) # intercept
    b_trait = np.zeros(num_generations) # slope
    z_trait = np.zeros(num_generations) # slope
    # Population Size Array
    population_size = np.zeros(num_generations)

    # Fitness Parameters
    mean_fitness = np.zeros(num_generations-1)
    cost = np.zeros(num_generations-1)
    fitBar = np.zeros(num_generations-1)

    x_trait[0] = x_init
    b_trait[0] = b_init
    population_size[0] = pop_size_init
    
    generations = 0

    for t in range(0,num_generations-1):
        if v is not None:
            eta = (v * t)
        elif v_array is not None:
            eta = v_array[t]
        theta = beta * eta


        if t == 0:
            z = x_trait[t] + b_trait[0] * eta
            z_trait[t] = z


        if population_size[t] <= 0:
            break
        generations += 1
        ##### Calculation Population
        ##### Changes
        rZero = r0**( 1 - (population_size[t] / K) )

        # cBar Calculations
        gammaB = 1/( (omegaB) + (sigmaB) )

        cBar = np.sqrt( gammaB*(omegaB) ) * ( np.exp( -(gammaB / 2) * b_trait[0]**2) )
        cost[t] = cBar
        # wBar Calculations

        # Assemble the Stars
        kappa = gammaB*sigmaB
        bStar = (1-kappa)*b_trait[t]
        sigmaBstar = (1-kappa) * sigmaB
        sigmaZstar = sigmaX + ( sigmaB * (eta**2) ) + sigmaE

        #### Expressed  Trait #####
        zStar = x_trait[t] + bStar * eta
        #### Possible Change to conditional based on limited range ####

        gammaZ = 1/( (omegaZ) + (sigmaZstar) ) 

        wBar = np.sqrt( gammaZ * (omegaZ) ) * np.exp( -(gammaZ/2) * (zStar - theta)**2 )
        fitBar[t] = wBar
        # rBar Calculations

        rBar = rZero*wBar*cBar
        mean_fitness[t] = rBar

        # Assigning New Population Size
        population_size[t+1] = math.floor(rBar * population_size[t])

        ##### Calculation Mean Trait ######
        ##### Changes

        # Change in intecept quantity 
        dx = -gammaZ * ( ( x_trait[t] + (bStar * theta) ) - (beta * theta) ) * sigmaX

        # Change in slope quantity 
        db = bStar - b_trait[t] - ( sigmaBstar * ( gammaZ * theta ) * ( x_trait[t] + (bStar * theta) - (beta * theta) ) )

        # Change in mean phenotype

        x_trait[t+1] = x_trait[t] + dx

        b_trait[t+1] = b_trait[t] + db

        zStar_mean = x_trait[t] + (bStar*eta)
        z_trait = zStar_mean
    return population_size, mean_fitness, x_trait, b_trait, generations, cost, fitBar